In [ ]:
from argparse import Action
from tkinter import E
from turtle import shape
from aiohttp import request
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_checker import check_env


import numpy as np
import random
import os
import win32api, win32con, win32gui
from PIL import ImageGrab
import time
import requests
import PIL
from PIL import Image
import mss
from matplotlib import pyplot as plt
import re
import cv2
from scipy.ndimage import convolve

In [ ]:





def make_a_play(board, play):
    board[play[0]][play[1]][play[2]] = 1
    return board


def get_reward(info, lenght_to_win):
    reward = 0
    for i in info:
        reward += (i-1)
    reward = (reward**(lenght_to_win))/((lenght_to_win)**(lenght_to_win))
    return reward


def make_horizontal_win_board(lenght_to_win):
    reward_board = [[]]
    for x in range(lenght_to_win):
        reward_board[0].append(1)
    return reward_board

def make_vertical_win_board(lenght_to_win):
    reward_board = []
    for y in range(lenght_to_win):
        reward_board.append([1])
    return reward_board

def make_left_up_diagonal_win_board(lenght_to_win):
    reward_board = []
    for y in range(lenght_to_win):
        reward_board.append([])
        for x in range(lenght_to_win):
            if y==x:
                reward_board[y].append(1)
            else:
                reward_board[y].append(0)
    return reward_board

def make_right_up_diagonal_win_board(lenght_to_win):
    reward_board = make_left_up_diagonal_win_board(lenght_to_win)
    reward_board = np.flipud(reward_board)
    return reward_board


def check_for_win(board,lenght_to_win):
    win_conditions = [make_vertical_win_board(lenght_to_win),make_horizontal_win_board(lenght_to_win),make_left_up_diagonal_win_board(lenght_to_win),make_right_up_diagonal_win_board(lenght_to_win)]
    for win_board in win_conditions:
        is_win = convolve(board,win_board,mode="constant")
        for y in is_win:
            for x in y:
                if x == lenght_to_win:
                    return True
        
    return False

def move_is_legal(playing_boards, play):
    if playing_boards[0][play[1]][play[2]] == 0 and playing_boards[1][play[1]][play[2]] == 0:
        return True
    else:
        return False
    
def is_draw(playing_boards,board_size):
    if np.sum(playing_boards) == board_size**2:
        return True
    else:
        return False
    
def flat_to_poss_in2d_array(n,size,player):
    y = int((n)/size)
    x = (n)- y*size
    action = [player,y,x]
    return action


    
def all_posible_moves(player,boards):
    posible_moves = []
    for y_index, y in enumerate(boards[0]):
        for x_index, x in enumerate(y):
            if boards[0][y_index][x_index] == 0 and boards[1][y_index][x_index] == 0:
                posible_moves.append([player,y_index,x_index])
    return posible_moves



def minmax(boards, depth = 0):
    best_move = None
    score_of_best_move = 0
    for posible_move in all_posible_moves(boards):
        boards[posible_move[0]][posible_move[1]][posible_move[2]] = 1
        player_win = check_for_win(boards[0],len(boards[0]))
        oponent_win = check_for_win(boards[0],len(boards[0]))
        draw = is_draw(boards[0],len(boards[0]))
        if best_move

        boards[posible_move[0]][posible_move[1]][posible_move[2]] = 0
        
        
    
    depth += 1
    return minmax()


In [ ]:
tim = []
class pyz_driver(Env):
    def __init__(self):
        self.first_to_play = True
        self.play_against_random = 0
        
        #setup the game
        self.board_size = 4
        self.lenght_to_win = 4
        self.playing_boards = np.array([np.zeros((self.board_size,self.board_size)), np.zeros((self.board_size,self.board_size))])
        self.playing_boards = self.playing_boards.astype(int)

        self.action_space = Box(-1, 1, shape=(self.board_size**2, ),dtype=np.float32)

        self.observation_space = Box(0,1, shape=(2,self.board_size,self.board_size),dtype=np.uint8)

        self.state = self.playing_boards
        self.boards = [[],[]]
        
        
        

    def do_action(self,action):   
        self.playing_boards[action[0]][action[1]][action[2]] = 1

    def step(self, action):
        reward = 0
        start = time.time()
        


        action = action.argmax()
        action = flat_to_poss_in2d_array(action,self.board_size,player=0)
        if move_is_legal(self.playing_boards,action):
            self.do_action(action)
            #reward += 1
        else:
            reward -= 1
            pass
        

        
        player_win = check_for_win(self.playing_boards[0],self.lenght_to_win)
        oponent_win = check_for_win(self.playing_boards[1],self.lenght_to_win)
        draw = is_draw(self.playing_boards,self.board_size)

        if player_win or oponent_win or draw:
            done = True
        else:
            done = False
        



        if not done:
            if self.play_against_random <2:
                posible_moves = all_posible_moves(player=1,boards=self.playing_boards)
                oponent_action = random.choice(posible_moves)
                self.do_action(oponent_action)
            else:
                rewersed_boards = np.array([self.playing_boards[1],self.playing_boards[0]])
                oponent_action = model.predict(rewersed_boards)
                oponent_action = oponent_action[0] # obasuhe tuple s vísledkem a NONE
                oponent_action = oponent_action.argmax()
                oponent_action = flat_to_poss_in2d_array(oponent_action,self.board_size,player=1)
                if move_is_legal(self.playing_boards,oponent_action):
                    self.do_action(oponent_action)
            

            player_win = check_for_win(self.playing_boards[0],self.lenght_to_win)
            oponent_win = check_for_win(self.playing_boards[1],self.lenght_to_win)
            draw = is_draw(self.playing_boards,self.board_size)

            if player_win or oponent_win or draw:
                done = True
            else:
                done = False




        if player_win:
            reward += 1#self.board_size**2
            self.boards[0].append(self.playing_boards)
            pass

        if oponent_win:
            reward -= 1
            self.boards[1].append(self.playing_boards)
            pass

        if draw and not(oponent_win or player_win):
            reward += 0.75
            pass

        if draw:
            #reward += 10
            pass

        
        tim.append(time.time()-start)
        info = {}
        return self.playing_boards , reward, done, info


    def render(self):
        print(self.playing_boards)

    def reset(self):
        self.playing_boards = np.array([np.zeros((self.board_size,self.board_size)), np.zeros((self.board_size,self.board_size))])
        self.playing_boards = self.playing_boards.astype(int)
        self.first_to_play = not self.first_to_play
        
        self.play_against_random += 1
        if self.play_against_random == 4:
            self.play_against_random = 0
        
        
        if not self.first_to_play:
            if self.play_against_random <2:
                posible_moves = all_posible_moves(player=1,boards=self.playing_boards)
                oponent_action = random.choice(posible_moves)
                self.do_action(oponent_action)
            else:
                rewersed_boards = np.array([self.playing_boards[1],self.playing_boards[0]])
                oponent_action = model.predict(rewersed_boards)
                oponent_action = oponent_action[0] # obasuhe tuple s vísledkem a NONE
                oponent_action = oponent_action.argmax()
                oponent_action = flat_to_poss_in2d_array(oponent_action,self.board_size,player=1)
                if move_is_legal(self.playing_boards,oponent_action):
                    self.do_action(oponent_action)
        
        
        
        return self.playing_boards

In [ ]:
from stable_baselines3.common.callbacks import BaseCallback
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
env = pyz_driver()
dir_id = time.time()
learning_rate = 0.00005
timesteps = 10_000_000
info = f"random_w_m_{env.board_size}x{env.board_size}_{env.lenght_to_win}_to_win_lr={learning_rate}"

CHECKPOINT_DIR = f'./train/{timesteps}_timesteps_{info}_{dir_id}'
LOG_DIR = f'./logs/{timesteps}_timesteps_{info}_{dir_id}'

callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [ ]:


model = PPO("MlpPolicy", env, tensorboard_log=LOG_DIR, verbose=1, learning_rate=learning_rate)
#check_env(env)
#model = PPO.load(r"train\2000000_timesteps_random_w_m_3x3_3_to_win_lr=5e-05_1681116393.3547344\best_model_2000000.zip", env=env)



In [ ]:

model.learn(total_timesteps=(timesteps), callback=callback)

In [ ]:
print(sum(tim)/len(tim))